# ETL Pipeline

In [ ]:
import importlib
from pathlib import Path

import pandas as pd

import helpers
importlib.reload(helpers)
from helpers import create_df, match_string

import utils
importlib.reload(utils)
from utils import logger

pd.set_option('display.float_format', lambda x: '%.2f' % x)

account_positions_csv_file = Path().cwd().parent / 'data' / 'account_positions.csv'
accounts_csv_file = Path().cwd().parent / 'data' / 'accounts.csv'
price_history_csv_file = Path().cwd().parent / 'data' / 'price_history.csv'
transactions_csv_file = Path().cwd().parent / 'data' / 'transactions.csv'

logger.info('ETL pipeline started...')

## Fact: Transaction

In [ ]:
# Create fact transactions df.
logger.info(f'Loading transactions from {transactions_csv_file}...')
transactions = pd.read_csv(transactions_csv_file)

In [ ]:
# Drop duplicates & rows with nan or null values.
transactions_before = len(transactions)
fact_transaction = create_df('fact_transaction', transactions, list(transactions.columns), 'transaction')
fact_transaction = fact_transaction.drop_duplicates().dropna()
transactions_after = len(fact_transaction)
logger.info(f'{transactions_before - transactions_after} duplicates/rows with nan or null values dropped.')

In [ ]:
# Fix typos in trade type column
# Define regex and apply boolean mask for each trade type to replace typos.
buy_regex = r'[buy]'
sell_regex = r'[sell]'

buy_mask = fact_transaction['trade_type'].str.contains(buy_regex, case=False, na=False)
sell_mask = fact_transaction['trade_type'].str.contains(sell_regex, case=False, na=False)

fact_transaction.loc[buy_mask, 'trade_type'] = 'buy'
fact_transaction.loc[sell_mask & (~buy_mask), 'trade_type'] = 'sell'

logger.info('Typos in trade type column fixed.')

In [ ]:
# Convert trade type to 'b' for 'buy' or 's' for 'sell.'
fact_transaction['trade_type'] = fact_transaction['trade_type'].replace({'buy': 'b', 'sell': 's'})
logger.info('Trade types replaced with single characters.')

In [ ]:
# Convert columns to proper dtypes.
fact_transaction = fact_transaction.astype({
    'transaction_id': 'int32',
    'account_id': 'int32',
    'ticker_symbol': 'string',
    'trade_type': 'category',
    'shares': 'float64',
    'price': 'float64'
})

In [ ]:
fact_transaction.head()

# TODO
#   Create dimension ticker and date tables then map ids to these columns.

## Fact: Account

In [ ]:
logger.info(f'Loading accounts from {accounts_csv_file}...')
accounts = pd.read_csv(accounts_csv_file)

In [ ]:
# Create dim_account.
dim_account = create_df('dim_account', accounts, ['account_id', 'user_name', 'first_name', 'last_name', 'account_type', 'created_at'])

In [ ]:
dim_account = dim_account.drop_duplicates()

In [ ]:
dim_account.describe(include='all')